In [1]:
# import laspy
import json
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc, col, round
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
import pandas as pd
import os
import sys
import laspy

Classification value	Meaning
0 Never classified

1 Unassigned

2 Ground

3 Low Vegetation

4 Medium Vegetation

5 High Vegetation

6 Building

7 Low Point

8 Reserved

9 Water

10 Rail

11 Road Surface

12 Reserved

13 Wire - Guard (Shield)

14 Wire - Conductor (Phase)

15 Transmission Tower

16 Wire-Structure Connector (Insulator)

17 Bridge Deck

18 High Noise

# Create Snowflake Connections and Stages

In [10]:
snowflake_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/creds.json").read())
aws_connection_cfg = json.loads(open("/Users/mitaylor/Documents/creds/aws_creds.json").read())

AWS_SECRET_KEY = aws_connection_cfg['password']
AWS_KEY_ID = aws_connection_cfg['account']
# Creating Snowpark Session
session = Session.builder.configs(snowflake_connection_cfg).create()

# Create a virtual warehouse, db and a stage for our ML models
session.sql("CREATE OR REPLACE WAREHOUSE LAZ_VH WITH WAREHOUSE_SIZE='X-SMALL'").collect()
session.sql("USE DATABASE LAZ_DB").collect()
#session.sql("CREATE OR REPLACE DATABASE LAZ_DB").collect()
#session.sql("CREATE OR REPLACE STAGE PIPELINE").collect()

[Row(status='Statement executed successfully.')]

## Create External Stage

In [59]:
# Create our external stages (in S3)
session.use_schema('PUBLIC')
session.sql(f"""
CREATE OR REPLACE STAGE LAZ_S3_STAGE 
URL = 's3://mtaylor-raw-data-store/laz'
CREDENTIALS = (AWS_KEY_ID = '{AWS_KEY_ID}'  AWS_SECRET_KEY = '{AWS_SECRET_KEY}')
file_format = (type = 'CSV' field_delimiter = ',');
""").collect()

[Row(status='Stage area LAZ_S3_STAGE successfully created.')]

### and check what's in it...

# Read and Clean LAZ File locally, just to see what we're dealing with

In [4]:
#Read LAS file
inFile = laspy.read('/Users/mitaylor/Documents/Sandbox/Zurich/LIDAR/LAS_FILES/TQ2080_P_9983_20150206_20150206.laz')

#Import LAS into numpy array (X=raw integer value x=scaled float value)
lidar_points = np.array((inFile.x,inFile.y,inFile.z,inFile.intensity,
               inFile.raw_classification,inFile.scan_angle_rank)).transpose()

#Transform to pandas DataFrame
lidar_df=pd.DataFrame(lidar_points, columns=["x","y","z","intensity","classification", "scan_angle_rank"])
lidar_df

,x,y,z,intensity,classification,scan_angle_rank
0,520817.34,181342.31,33.75,10.0,8.0,-22.0
1,520817.15,181343.70,34.80,94.0,4.0,-22.0
2,520817.15,181344.25,33.79,52.0,2.0,-22.0
3,520817.06,181345.25,33.87,60.0,8.0,-22.0
4,520816.98,181346.20,33.82,62.0,3.0,-22.0
...,...,...,...,...,...,...
3461446,521998.15,181999.46,23.68,54.0,4.0,4.0
3461447,521998.96,181999.57,29.13,15.0,5.0,4.0
3461448,521999.68,181998.93,30.49,15.0,5.0,4.0
3461449,521999.50,181999.47,23.61,22.0,4.0,4.0


## And fix the easting Northing "Issue"

In [5]:
from pyproj import Proj, transform
import pandas as pd


v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy",
        towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
vgrid = Proj(init="world:bng")

def vectorized_convert(df):
    vlon36, vlat36 = vgrid(df['x'].values, 
                           df['y'].values, 
                           inverse=True)
    converted = transform(v36, v84, vlon36, vlat36)
    df['longitude'] = converted[0]
    df['latitude'] = converted[1]
    return df

vectorized_convert(lidar_df)

/Users/mitaylor/opt/anaconda3/envs/snowpark_gis/lib/python3.8/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/var/folders/97/8vc6xcbx4zd06p75xg9frdrw0000gn/T/ipykernel_27606/2713558150.py:14: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  converted = transform(v36, v84, vlon36, vlat36)


,x,y,z,intensity,classification,scan_angle_rank,longitude,latitude
0,520817.34,181342.31,33.75,10.0,8.0,-22.0,-0.260130,51.518090
1,520817.15,181343.70,34.80,94.0,4.0,-22.0,-0.260132,51.518103
2,520817.15,181344.25,33.79,52.0,2.0,-22.0,-0.260132,51.518108
3,520817.06,181345.25,33.87,60.0,8.0,-22.0,-0.260133,51.518117
4,520816.98,181346.20,33.82,62.0,3.0,-22.0,-0.260134,51.518125
...,...,...,...,...,...,...,...,...
3461446,521998.15,181999.46,23.68,54.0,4.0,4.0,-0.242893,51.523743
3461447,521998.96,181999.57,29.13,15.0,5.0,4.0,-0.242881,51.523743
3461448,521999.68,181998.93,30.49,15.0,5.0,4.0,-0.242871,51.523738
3461449,521999.50,181999.47,23.61,22.0,4.0,4.0,-0.242874,51.523742


In [6]:
pd.DataFrame(session.sql('LS @my_s3_train_stage').collect())

,name,size,md5,last_modified
0,s3://mtaylor-raw-data-store/laz/TQ2080_P_9983_...,12135892,160f236219c5b34354508702cde549a2,"Mon, 31 Jul 2023 14:25:11 GMT"
1,s3://mtaylor-raw-data-store/laz/TQ2082_P_9983_...,11113655,4dba0ace4ccbfa21f825fda13b2f3a8c,"Mon, 31 Jul 2023 14:27:51 GMT"
2,s3://mtaylor-raw-data-store/laz/TQ2084_P_9983_...,552357,c89abbdf9da9e25a2b232210447b436f,"Mon, 31 Jul 2023 14:27:31 GMT"
3,s3://mtaylor-raw-data-store/laz/TQ2280_P_9983_...,7218310,664d0c2b9963ba7d32ecc2fdd983ea92,"Mon, 31 Jul 2023 14:27:16 GMT"
4,s3://mtaylor-raw-data-store/laz/TQ2282_P_9983_...,35108512,fb084d58a1acc43f890fffb8d54c2f45-3,"Mon, 31 Jul 2023 14:25:07 GMT"
5,s3://mtaylor-raw-data-store/laz/TQ2284_P_9983_...,28803653,2b3c1ab85886e00b32c421be1c986d61-2,"Mon, 31 Jul 2023 14:25:07 GMT"
6,s3://mtaylor-raw-data-store/laz/TQ2286_P_9983_...,4307912,1a092a0fd37d105f377f55752998c0d1,"Mon, 31 Jul 2023 14:26:17 GMT"
7,s3://mtaylor-raw-data-store/laz/TQ2480_P_9983_...,1785829,212a25b22399d38bdb196017775a2e6a,"Mon, 31 Jul 2023 14:25:54 GMT"
8,s3://mtaylor-raw-data-store/laz/TQ2482_P_9983_...,32291531,5f90b411dc664c6c47307c240c560de3-2,"Mon, 31 Jul 2023 14:25:07 GMT"
9,s3://mtaylor-raw-data-store/laz/TQ2484_P_9983_...,38356034,57f4e2ee6d8a614bfca178928d2df930-3,"Mon, 31 Jul 2023 14:25:07 GMT"


# Create a Sproc to do the same, but loop over everything in the db

TLDR: 30 files takes 25mins on an extra small warehouse, approximately 0.015 credits per Laz file.  Depending on deal structure 1 credit equates to approximately 5 dollars, so each file costs c. 0.07 dollars to read, transform and load.

Note this ingestion can easily be parallelised to expedite ingestion, but the compute estimates would remain similar without optimisations, and it is likley that multiple async calls to the same XS warehouse could be sustained, which could reduce costs by 50% or more

This reflects 0.75GB of data, i.e. c. 1300 times less data than the presumed TB of LAZ files

Assume 1500x more files, XS warehouse will take 25mins x 1500 = 600 hours or c. 25 days.  
XS Warehouse has 1/32 the compute of an 2XL Warehouse, so 2XL completes in c. 0.75 day, at a cost of c. 600 Credits based on current performance with any optimisations (more parallelisation is expected to be possible for example)

XS = 1
S  = 2
M  = 4
L  = 8
XL = 16
2XL= 32

It's inefficient as I've just done it in a single SPROC, using a UDTF will be faster as it can fire off lots of sprocs, but that will be my next job

In [9]:
@sproc(name='import_laz', 
       packages=['snowflake-snowpark-python','laspy', 'lazrs-python', 'pyproj'], 
       is_permanent=True, 
       replace=True,
       stage_location='@PIPELINE', 
       session=session)
def read_laz(session: Session, df_iloc_start: int, df_iloc_end: int) -> T.Variant:
    from snowflake.snowpark.files import SnowflakeFile
    import pandas as pd
    from pyproj import Proj, transform
    import pandas as pd
    
    
    v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
    v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy",
            towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
    vgrid = Proj(init="world:bng")
    
    def vectorized_convert(df):
        vlon36, vlat36 = vgrid(df['x'].values, 
                               df['y'].values, 
                               inverse=True)
        converted = transform(v36, v84, vlon36, vlat36)
        df['longitude'] = converted[0]
        df['latitude'] = converted[1]
        return df
    
    
    scope = pd.DataFrame(session.sql('LS @LAZ_S3_STAGE').collect())
    scope = scope.iloc[df_iloc_start:df_iloc_end,:]
    
    for row in range(len(scope)):
        raw_url = scope.iloc[row][0].split('/laz')[1]
        scoped_url = session.sql(f"SELECT BUILD_SCOPED_FILE_URL( @MY_S3_TRAIN_STAGE , '{raw_url}')").collect()[0][0]
        with SnowflakeFile.open(scoped_url, 'rb') as f:
            inFile = laspy.read(f)  
            #Import LAS into numpy array (X=raw integer value x=scaled float value)
            lidar_points = np.array((inFile.x,inFile.y,inFile.z,inFile.intensity,
                           inFile.raw_classification,inFile.scan_angle_rank)).transpose()
            
            #Transform to pandas DataFrame
            lidar_df=pd.DataFrame(lidar_points, columns=["x","y","z","intensity","classification", "scan_angle_rank"])

            #Update with Long/Lat
            lidar_df = vectorized_convert(lidar_df)
            
            sdf = session.create_dataframe(lidar_df)
            sdf.write.save_as_table("LIDAR_DATASET", mode="append") # do this async?
            #session.write_pandas(lidar_df, table_name='LIDAR_DATASET', auto_create_table=True, overwrite=True)
            
    return ("LAZ FILES INGESTED")

Package 'lazrs-python' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.


In [8]:
%%time
read_laz(0,30)

CPU times: user 388 ms, sys: 88.8 ms, total: 477 ms
Wall time: 24min 4s


'"LAZ FILE INGESTED"'

In [10]:
%%time
read_laz(0,15)

CPU times: user 163 ms, sys: 36.1 ms, total: 199 ms
Wall time: 10min


'"LAZ FILE INGESTED"'

In [11]:
%%time
session.sql("CALL import_laz(0,15)").collect(block=False)
session.sql("CALL import_laz(15,30)").collect(block=False)


CPU times: user 40.3 ms, sys: 3.05 ms, total: 43.4 ms
Wall time: 231 ms


In [12]:
session.sql("CALL import_laz(0,15)").collect(block=False)

In [13]:
session.sql("CALL import_laz(15,30)").collect(block=False)


# Create a proper coordinate so we can query it efficiently with polygons

In [7]:
snowpark_df_gis = session.sql('''SELECT "x","y","z","intensity","classification", "scan_angle_rank", "longitude", "latitude", ST_MAKEPOINT(o."longitude", o."latitude") as COORD
                               FROM LIDAR_DATASET o''')
snowpark_df_gis.write.mode("overwrite").save_as_table("LIDAR_DATASET_GIS")

In [8]:
snowpark_df_gis.limit(5).to_pandas()

,x,y,z,intensity,classification,scan_angle_rank,longitude,latitude,COORD
0,521917.79,181566.25,19.35,23.0,2.0,-5.0,-0.244201,51.519867,"{\n ""coordinates"": [\n -2.442005173434572e..."
1,521918.20,181565.75,22.32,27.0,4.0,-5.0,-0.244195,51.519862,"{\n ""coordinates"": [\n -2.441947838449080e..."
2,521918.47,181565.71,19.35,14.0,2.0,-5.0,-0.244191,51.519862,"{\n ""coordinates"": [\n -2.441909081454464e..."
3,521918.87,181565.20,22.49,18.0,4.0,-5.0,-0.244185,51.519857,"{\n ""coordinates"": [\n -2.441853221613215e..."
4,521919.17,181565.16,19.27,11.0,2.0,-5.0,-0.244181,51.519857,"{\n ""coordinates"": [\n -2.441810142930061e..."


## And query with a crude coordinate

In [9]:
snowpark_df_gis_test = session.sql('''
SELECT "intensity","classification", "scan_angle_rank", "longitude", "latitude", o.COORD,  ST_CONTAINS(nl_poly, o.coord) as in_n_london
FROM LIDAR_DATASET_GIS o, (SELECT(TO_GEOGRAPHY('POLYGON((51.5 0, 52 0.5, 51.5 1, 51.5 0))')) AS nl_poly)''')

In [10]:
snowpark_df_gis_test.limit(5).to_pandas()

,intensity,classification,scan_angle_rank,longitude,latitude,COORD,IN_N_LONDON
0,66.0,1.0,-14.0,-0.241385,51.519131,"{\n ""coordinates"": [\n -2.413847029169831e...",False
1,78.0,1.0,-14.0,-0.241385,51.519139,"{\n ""coordinates"": [\n -2.413853893480410e...",False
2,81.0,3.0,-14.0,-0.241386,51.519148,"{\n ""coordinates"": [\n -2.413859282635153e...",False
3,79.0,1.0,-14.0,-0.241386,51.519156,"{\n ""coordinates"": [\n -2.413864775645290e...",False
4,70.0,1.0,-14.0,-0.241387,51.519164,"{\n ""coordinates"": [\n -2.413871605343854e...",False


# Test Out H3 and Performance (Locally)

In [11]:
N = 1_000_000

lats = np.random.uniform(0, 90, N) 
lons = np.random.uniform(0, 90, N)

### String Approach

In [12]:
import h3.api.basic_str as h3s

In [13]:
%%timeit
[h3s.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

9.33 s ± 11.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Numpy Approach

In [14]:
import h3.api.numpy_int as h3i

In [15]:
%%timeit
[h3i.geo_to_h3(lat, lon, 10) for lat, lon in zip(lats, lons)]

9.17 s ± 48.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Vectorised Approach
#### (note this is classed as unstable by the H3 devs, but prob stable given it's an old(er) version, now on 4.0 in beta)

In [16]:
from h3.unstable import vect

/Users/mitaylor/opt/anaconda3/envs/snowpark_gis/lib/python3.8/site-packages/h3/unstable/__init__.py:4: UserWarning: Modules under `h3.unstable` are experimental, and may change at any time.
  warnings.warn(


In [17]:
%%timeit
vect.geo_to_h3(lats, lons, 9)

8.49 s ± 37.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Probably/possibly a CPU issue, so this might have greater performance, but not a dramatic improvement as is

# Now Enrich Snowflake With H3

TLDR: with an XS warehouse this takes 21minutes for ~200M rows (c. 7GB), so c. 0.4 credits or around 2 dollars per 200M rows.  Scaling to a TB of data assume c. 3-400 dollars
this will scale linearly in cost and can be parallelised to reduce time of execution

In [27]:
feature_cols = ['"x"',
 '"y"',
 '"z"',
 '"intensity"',
 '"classification"',
 '"scan_angle_rank"',
 '"longitude"',
 '"latitude"']

def h3_conversion(df: pd.DataFrame) -> pd.Series:
    import pandas as pd
    df = pd.DataFrame(np.array(df), columns =  ['x','y', 'z', 'intensity', 'classification', 'scan_angle_rank', 'longitude', 'latitude'])   
    lons = np.array(df['longitude'])
    lats = np.array(df['latitude'])

    #import h3.api.numpy_int as h3i
    #hex = pd.Series([h3i.geo_to_h3(lat, lon, 5) for lat, lon in zip(lats, lons)])
    import h3.api.basic_str as h3s
    hex = pd.Series([h3s.geo_to_h3(lat, lon, 12) for lat, lon in zip(lats, lons)]) 
    # 12 is about 17.5m x 17.5m, appropriate for a house, 11 for a block of flats (50m by 50m ish), 10 for a very big building (100m by 100m ish)
    return hex

udf_model = session.udf.register(session=session,func=h3_conversion,
                                 name="h3_conversion", stage_location='@PIPELINE',
                                 input_types=[T.FloatType()]*len(feature_cols),
                                 return_type=T.StringType(),
                                 replace=True, is_permanent=True,
                                 max_batch_size=1000,
                                 packages=['h3-py', 'numpy'])

Package 'h3-py' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.
The version of package 'numpy' in the local environment is 1.24.2, which does not fit the criteria for the requirement 'numpy'. Your UDF might not work when the package version is different between the server and your local environment.


In [19]:
%%time
test_sdf = session.table('LIDAR_DATASET')
test_sdf = test_sdf.with_column('hex', udf_model(*feature_cols))
test_sdf.write.mode("overwrite").save_as_table("LIDAR_DATASET_HEX")

CPU times: user 350 ms, sys: 91.1 ms, total: 441 ms
Wall time: 21min 17s


In [20]:
test_sdf.limit(5).to_pandas()

,x,y,z,intensity,classification,scan_angle_rank,longitude,latitude,HEX
0,521917.79,181566.25,19.35,23.0,2.0,-5.0,-0.244201,51.519867,8c195da5bb59dff
1,521918.20,181565.75,22.32,27.0,4.0,-5.0,-0.244195,51.519862,8c195da5bb59dff
2,521918.47,181565.71,19.35,14.0,2.0,-5.0,-0.244191,51.519862,8c195da5bb59dff
3,521918.87,181565.20,22.49,18.0,4.0,-5.0,-0.244185,51.519857,8c195da5bb59dff
4,521919.17,181565.16,19.27,11.0,2.0,-5.0,-0.244181,51.519857,8c195da5bb59dff


## With PuPr features

In [11]:
%%time
test_sdf = session.sql('select H3_LATLNG_TO_CELL(lidar_dataset_gis."longitude", lidar_dataset_gis."latitude", 10) as HEX10, "z","intensity","classification", "scan_angle_rank" from lidar_dataset_gis')

CPU times: user 198 µs, sys: 5 µs, total: 203 µs
Wall time: 209 µs


In [12]:
%%time
test_sdf.write.mode("overwrite").save_as_table("LIDAR_DATASET_HEX_PUPR")

CPU times: user 89.4 ms, sys: 16.7 ms, total: 106 ms
Wall time: 3min 1s


In [13]:
session.sql("CREATE OR REPLACE WAREHOUSE LAZ_VH WITH WAREHOUSE_SIZE='X-LARGE'").collect()

[Row(status='Warehouse LAZ_VH successfully created.')]

In [14]:
%%time
test_sdf.write.mode("overwrite").save_as_table("LIDAR_DATASET_HEX_PUPR_XL")

CPU times: user 23.6 ms, sys: 2.99 ms, total: 26.6 ms
Wall time: 15.9 s


Using Native PuPr functions we're able to shave c. 90% of the execution time off, ie from 21mins to 2 mins, and we can further compress this with more Warehouse time (but this does not save money, just time)

# Simulate the Properties

In [5]:
props_pdf = session.table('LIDAR_DATASET_HEX').limit(5000000).to_pandas()
props_pdf = props_pdf.sample(n=100000,replace=True)
props_pdf['HEIGHT'] = props_pdf['z']
props_pdf = props_pdf[['HEX', 'HEIGHT']]
props_pdf

,HEX,HEIGHT
3089063,8c195da59ab65ff,18.21
4418569,8c195da58223dff,44.35
2622900,8c195da5c2f17ff,49.67
836255,8c195da591b63ff,36.20
2548997,8c195da5d2137ff,49.12
...,...,...
4107607,8c195da594dabff,21.25
2426980,8c195da5dc0d3ff,53.70
821464,8c195da590927ff,24.40
3136421,8c195da5baec1ff,13.70


##  Load the props

In [6]:
session.write_pandas(df=props_pdf,
                     table_name="PROP_TABLE",
                     overwrite=True,
                     auto_create_table=True)

/Users/mitaylor/opt/anaconda3/envs/snowpark_gis/lib/python3.8/site-packages/snowflake/snowpark/session.py:1766: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, ci_output = write_pandas(


## Get the Min Heights

In [7]:
sdf = session.sql('''SELECT MIN("z") as MIN_Z, HEX
  FROM LIDAR_DATASET_HEX 
  GROUP BY HEX
  ORDER BY HEX''')

In [8]:
sdf.write.mode("overwrite").save_as_table("LIDAR_DATASET_HEX_HEIGHTS")

In [9]:
sdf.limit(5).to_pandas()

,MIN_Z,HEX
0,17.85,8c194ad36bb2bff
1,18.14,8c194ad36bb2dff
2,17.91,8c194ad36bb31ff
3,17.89,8c194ad36bb33ff
4,18.03,8c194ad36bb35ff


# Join Together and Generate Height

In [10]:
joined_sdf = session.sql('''SELECT *
FROM LIDAR_DATASET_HEX_HEIGHTS
JOIN PROP_TABLE 
ON PROP_TABLE.HEX = LIDAR_DATASET_HEX_HEIGHTS.HEX''')

In [11]:
joined_sdf.limit(5).show()

---------------------------------------------------------------------
|"MIN_Z"             |"HEX"            |"HEX"            |"HEIGHT"  |
---------------------------------------------------------------------
|6.76                |8c194ada64049ff  |8c194ada64049ff  |7.04      |
|6.76                |8c194ada64049ff  |8c194ada64049ff  |19.07     |
|6.76                |8c194ada64049ff  |8c194ada64049ff  |8.33      |
|7.0200000000000005  |8c194ada6404dff  |8c194ada6404dff  |12.07     |
|7.0200000000000005  |8c194ada6404dff  |8c194ada6404dff  |12.94     |
---------------------------------------------------------------------



In [15]:
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType

In [16]:
class k_ring_finder:

    def __init__(self):
        None
        
    def end_partition(self, df):
        import h3
        df.columns = ["HEX", "HEIGHT", "K"]
        K = list(df['K'])[0]
        HEX = list(df['HEX'])[0]
        HEIGHT = list(df['HEIGHT'])[0]
        k_ring = list(h3.k_ring(HEX, K))
        df_new = pd.DataFrame()
        df_new['K_RING_HEX'] = k_ring
        df_new['K'] = K
        df_new['HEIGHT'] = HEIGHT
        df_new['ORIGIN_HEX'] = HEX
        yield df_new

k_ring_finder.end_partition._sf_vectorized_input = pd.DataFrame

In [17]:
k_ring_finder_udtf = session.udtf.register(
    k_ring_finder,
    input_types=[PandasDataFrameType([StringType(),
                                      FloatType(),
                                      IntegerType()]
                                    )],
    output_schema=PandasDataFrameType([StringType(), IntegerType(), FloatType(), StringType()], ["K_RING_HEX_", "K_", "HEIGHT_", "ORIGIN_HEX_"]),
    packages=['snowflake-snowpark-python', 'h3-py', 'numpy', 'pandas'])

Package 'h3-py' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.
The version of package 'numpy' in the local environment is 1.24.2, which does not fit the criteria for the requirement 'numpy'. Your UDF might not work when the package version is different between the server and your local environment.


In [20]:
from snowflake.snowpark.functions import col, lit
sdf_test = session.table('PROP_TABLE')
sdf_test = sdf_test.with_column('K', lit(1))
sdf_test.show()

----------------------------------------------
|"HEX"            |"HEIGHT"            |"K"  |
----------------------------------------------
|8c195da59ab65ff  |18.21               |1    |
|8c195da58223dff  |44.35               |1    |
|8c195da5c2f17ff  |49.67               |1    |
|8c195da591b63ff  |36.2                |1    |
|8c195da5d2137ff  |49.120000000000005  |1    |
|8c195da5b86d3ff  |26.44               |1    |
|8c195da5d6b17ff  |59.74               |1    |
|8c195da591821ff  |34.300000000000004  |1    |
|8c195da590f21ff  |26.22               |1    |
|8c195da5b1197ff  |46.64               |1    |
----------------------------------------------



In [21]:
sdf_test = sdf_test.select(k_ring_finder_udtf('hex', 'height', 'k').over(partition_by=['hex']))

In [22]:
sdf_test.limit(20).to_pandas()

,K_RING_HEX_,K_,HEIGHT_,ORIGIN_HEX_
0,8c195da5b04a3ff,1,33.34,8c195da5b04a1ff
1,8c195da5b04a1ff,1,33.34,8c195da5b04a1ff
2,8c195da5b04a5ff,1,33.34,8c195da5b04a1ff
3,8c195da5b04adff,1,33.34,8c195da5b04a1ff
4,8c195da5b04a9ff,1,33.34,8c195da5b04a1ff
5,8c195da5b04abff,1,33.34,8c195da5b04a1ff
6,8c195da5b04a7ff,1,33.34,8c195da5b04a1ff
7,8c195da5bb265ff,1,29.29,8c195da5bb265ff
8,8c195da5bb359ff,1,29.29,8c195da5bb265ff
9,8c195da5bb261ff,1,29.29,8c195da5bb265ff


In [23]:
sdf_test.write.mode("overwrite").save_as_table("PROP_TABLE_HEX_K")

In [24]:
%%time
joined_sdf = session.sql('''SELECT *
FROM LIDAR_DATASET_HEX_HEIGHTS
JOIN PROP_TABLE_HEX_K 
ON PROP_TABLE_HEX_K.K_RING_HEX_ = LIDAR_DATASET_HEX_HEIGHTS.HEX''')
joined_sdf.limit(20).to_pandas()

CPU times: user 26.7 ms, sys: 7.56 ms, total: 34.3 ms
Wall time: 823 ms


,MIN_Z,HEX,K_RING_HEX_,K_,HEIGHT_,ORIGIN_HEX_
0,6.76,8c194ada64049ff,8c194ada64049ff,1,12.07,8c194ada6404dff
1,6.76,8c194ada64049ff,8c194ada64049ff,1,6.96,8c194ada64227ff
2,6.76,8c194ada64049ff,8c194ada64049ff,1,16.42,8c194ada64223ff
3,6.76,8c194ada64049ff,8c194ada64049ff,1,21.16,8c194ada64235ff
4,6.76,8c194ada64049ff,8c194ada64049ff,1,7.04,8c194ada64049ff
5,6.75,8c194ada6404bff,8c194ada6404bff,1,21.16,8c194ada64235ff
6,6.75,8c194ada6404bff,8c194ada6404bff,1,7.04,8c194ada64049ff
7,7.02,8c194ada6404dff,8c194ada6404dff,1,12.07,8c194ada6404dff
8,7.02,8c194ada6404dff,8c194ada6404dff,1,6.96,8c194ada64227ff
9,7.02,8c194ada6404dff,8c194ada6404dff,1,7.04,8c194ada64049ff


process is as follows:

1. Get the location of any property into PROP_SDF
2. Enrich PROP_SDF with the H311 location of the property
3. Enrich PROP_SDF with the H311 "avg height"
4. Enrich/Expand PROP_SDF with the kring for each the H311 (just 1 for time being) <-- note the table should be 7 large because of the tesselation
5. PROP_SDF inner join LIDAR DATA (at H311) as FULL_TABLE
6. FULL_TABLE do group by on K_RING_ to get min heights added (and dump a bunch of cols while you're at it)
7. do the delta

# Scratch Pad

## WIP Parallelising Import Efficiently

In [84]:
from snowflake.snowpark.types import PandasDataFrameType, IntegerType, StringType, FloatType, DateType

class Ingest_Laz:
    """
    UDTF class to read, transform and load LAZ Files

    Yields
    -------
    lidar_df : DataFrame
        DataFrame with the "x","y","z","intensity","classification", "scan_angle_rank"

    """
    def __init__(self):
        None
        
    def end_partition(self, df):
        from snowflake.snowpark.files import SnowflakeFile
        import pandas as pd
        from pyproj import Proj, transform
        import pandas as pd
        v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
        v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy",
                towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
        vgrid = Proj(init="world:bng")

        def vectorized_convert(df):
            vlon36, vlat36 = vgrid(df['x'].values, 
                                   df['y'].values, 
                                   inverse=True)
            converted = transform(v36, v84, vlon36, vlat36)
            df['longitude'] = converted[0]
            df['latitude'] = converted[1]
            return df
        
        scoped_url = df.iloc[0][0]
       
        with SnowflakeFile.open(scoped_url, 'rb') as f:
            inFile = laspy.read(f)

            #Import LAS into numpy array (X=raw integer value x=scaled float value)
            lidar_points = np.array((inFile.x,inFile.y,inFile.z,inFile.intensity,
                           inFile.raw_classification,inFile.scan_angle_rank)).transpose()
            
            #Transform to pandas DataFrame
            lidar_df=pd.DataFrame(lidar_points, columns=["x_","y_","z_","intensity_","classification_", "scan_angle_rank_"])
    
            #Update with Long/Lat
            lidar_df = vectorized_convert(lidar_df)
        yield df # lidar_df
        
Ingest_Laz.end_partition._sf_vectorized_input = pd.DataFrame

laz_prep_udtf = session.udtf.register(
    handler=Ingest_Laz, # the class
    input_types=[PandasDataFrameType([StringType()])], 
    output_schema=PandasDataFrameType([StringType(),FloatType(),FloatType(),FloatType(),FloatType(),FloatType()],
                                      ["x","y","z","intensity","classification", "scan_angle_rank"]),
    packages=["snowflake-snowpark-python", 'pandas', 'laspy', 'lazrs-python', 'pyproj'])  

Package 'lazrs-python' is not installed in the local environment. Your UDF might not work when the package is installed on the server but not on your local environment.


In [85]:
sdf = session.sql('LS @LAZ_S3_STAGE')
sdf.write.save_as_table("URLS", mode="append")
session.sql("""CREATE OR REPLACE SECURE VIEW LAZ_FILES
AS
  SELECT BUILD_SCOPED_FILE_URL(@LAZ_S3_STAGE, '"name"') scoped_url
  FROM URLS;""").collect()

[Row(status='View LAZ_FILES successfully created.')]

In [86]:
session.sql('LS @LAZ_S3_STAGE').show()


--------------------------------------------------------------------------------------------------------------------------------------
|"name"                                              |"size"    |"md5"                               |"last_modified"                |
--------------------------------------------------------------------------------------------------------------------------------------
|s3://mtaylor-raw-data-store/laz/TQ2080_P_9983_2...  |12135892  |160f236219c5b34354508702cde549a2    |Mon, 31 Jul 2023 14:25:11 GMT  |
|s3://mtaylor-raw-data-store/laz/TQ2082_P_9983_2...  |11113655  |4dba0ace4ccbfa21f825fda13b2f3a8c    |Mon, 31 Jul 2023 14:27:51 GMT  |
|s3://mtaylor-raw-data-store/laz/TQ2084_P_9983_2...  |552357    |c89abbdf9da9e25a2b232210447b436f    |Mon, 31 Jul 2023 14:27:31 GMT  |
|s3://mtaylor-raw-data-store/laz/TQ2280_P_9983_2...  |7218310   |664d0c2b9963ba7d32ecc2fdd983ea92    |Mon, 31 Jul 2023 14:27:16 GMT  |
|s3://mtaylor-raw-data-store/laz/TQ2282_P_9983_2...  |3

In [87]:
sdf_final = session.table("LAZ_FILES")
sdf_final.limit(5).to_pandas()

,SCOPED_URL
0,https://be74032.eu-west-2.aws.snowflakecomputi...
1,https://be74032.eu-west-2.aws.snowflakecomputi...
2,https://be74032.eu-west-2.aws.snowflakecomputi...
3,https://be74032.eu-west-2.aws.snowflakecomputi...
4,https://be74032.eu-west-2.aws.snowflakecomputi...


In [88]:
%%time
all_cols = sdf_final.columns
sdf_prepped = sdf_final.select(laz_prep_udtf(*all_cols).over(partition_by=["SCOPED_URL"]))
sdf_prepped.limit(5).to_pandas()

SnowparkSQLException: (1304): 01afc7a4-0000-9f64-0000-f1490048372a: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 4, in <module>
  File "/usr/lib/python_udf/ba8972a674112c44f5ff7919df4637aff7052f43f0850f500ce5585cded9a06b/lib/python3.8/site-packages/cloudpickle/cloudpickle.py", line 679, in subimport
    __import__(name)
  File "/usr/lib/python_udf/ba8972a674112c44f5ff7919df4637aff7052f43f0850f500ce5585cded9a06b/lib/python3.8/site-packages/laspy/__init__.py", line 6, in <module>
    from .copc import CopcReader, Bounds
  File "/usr/lib/python_udf/ba8972a674112c44f5ff7919df4637aff7052f43f0850f500ce5585cded9a06b/lib/python3.8/site-packages/laspy/copc.py", line 34, in <module>
    DEFAULT_HTTP_WORKERS_NUM = multiprocessing.cpu_count() * 5
  File "/usr/lib/python_udf/ba8972a674112c44f5ff7919df4637aff7052f43f0850f500ce5585cded9a06b/lib/python3.8/multiprocessing/context.py", line 45, in cpu_count
    raise NotImplementedError('cannot

In [ ]:
Blog

Local

SPROC

Async SPROC

UDTF

Performance comparisons

https://medium.com/@fabian.gampfer/scaling-pycaret-on-snowflake-snowpark-for-python-91b723d5f1ff

1. S3 access
2. Python access (done, but maybe on a local jupyter/vscode)
3. Import LAZ to snowflake single thread initially (Mike to sort Async calls)
4. H3 grid for all(?) levels, Isaac to confirm appropriate levels
5. Additional Table - Property ID, Long, Lat, centroid, ------we need to create----> HexIDs (at each level?)

first assumption take the centroid and 3m hex(12-13), krings around until we reach a different terrain


build big kring, take bottom 20% of height vs top 20% height, delta is the height

from that  we can say the hexs for each property (see 5. above)

6. we have prop boundary, so we can build prop height (min/max/median)